<a href="https://colab.research.google.com/github/marioalbertod/ACP/blob/master/parse_xml_papers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET

In [ ]:
!wget -O 1.xml https://www.dropbox.com/s/ahcw6rdh45kwkyl/1.xml?dl=1

--2020-06-18 01:06:52--  https://www.dropbox.com/s/ahcw6rdh45kwkyl/1.xml?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.1, 2620:100:601d:1::a27d:501
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/ahcw6rdh45kwkyl/1.xml [following]
--2020-06-18 01:06:52--  https://www.dropbox.com/s/dl/ahcw6rdh45kwkyl/1.xml
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4975e8b1adab6f316e591ecf2a.dl.dropboxusercontent.com/cd/0/get/A51g5mv8K61RDjLRaI0swZe27OdSKwzq5Vd9kny3Ls4EE4Rj6-ywr0SBYxCqiCZDV6WVeeYnnDnKtGcCq18NtnXoAxZ0buXKtN0yJY9I-txS-OcV7I4H06MkS6KoElqW3_A/file?dl=1# [following]
--2020-06-18 01:06:52--  https://uc4975e8b1adab6f316e591ecf2a.dl.dropboxusercontent.com/cd/0/get/A51g5mv8K61RDjLRaI0swZe27OdSKwzq5Vd9kny3Ls4EE4Rj6-ywr0SBYxCqiCZDV6WVeeYnnDnKtGcCq18NtnXoAxZ0buXKtN0yJY9I-txS-OcV7I4H06MkS6KoElqW3_A/f

In [ ]:
root = ET.parse('1.xml').getroot()

Cuando lees el xml como un arbol pudes buscar un nodo y acceder a todo los hijos de este. A su vez puedes hacer nuevas busquedas sobre cada uno de esos hijos.

En la celda de abajo puedes ver que al iterar sobre la raiz del arbol puedes acceder a todos los hijos, en particuar el que nosotros interesa que es **back**

In [ ]:
for k in root:
    print(k)

<Element 'front' at 0x7f54fb273b38>
<Element 'body' at 0x7f54fb27edb8>
<Element 'body' at 0x7f54fb27ee08>
<Element 'body' at 0x7f54fb27ee58>
<Element 'body' at 0x7f54fb27eef8>
<Element 'body' at 0x7f54fb27ef48>
<Element 'body' at 0x7f54fb27ef98>
<Element 'body' at 0x7f54fb285048>
<Element 'body' at 0x7f54fb2850e8>
<Element 'body' at 0x7f54fb285188>
<Element 'body' at 0x7f54fb285228>
<Element 'body' at 0x7f54fb285278>
<Element 'body' at 0x7f54fb2852c8>
<Element 'body' at 0x7f54fb285318>
<Element 'body' at 0x7f54fb2853b8>
<Element 'body' at 0x7f54fb285458>
<Element 'body' at 0x7f54fb2854f8>
<Element 'body' at 0x7f54fb285548>
<Element 'body' at 0x7f54fb285598>
<Element 'body' at 0x7f54fb2855e8>
<Element 'back' at 0x7f54fb285638>


Otra de las cosas que se puede hacer es tratar los nodos como si fueran directorios de archivos del estilo _c:/User/my Documents/Music_, y buscar en una carpeta (nodo) especifico. Los pasos que segui para obtener la data que necesitas de las referencas es:
1. paso 1: buscar todos los nodos que son hijos de _'back/ref-list/ref/nlm-citation'_ e iterar por cada uno de los hijos de estos, de donde vamos a sacar la infomacion que necesitamos. Con estos nodos hijos lo que vamos a hacer es hacer una busqueda especifica dentro de ese nodo para sacar la info que necesitamos
2. por cada nodo hijo podemos hacer una busqueda especifica, por ejemplo el nodo 'year' y sacar el texto que este tiene 
3. por cada nodo hijo podemos hacer una busqueda especifica, por ejemplo el nodo 'citetion-type' y sacar el algun atributo que tenga este     
`<nlm-citation citetion-type='blablabla'>`
4. para cada nodo hijo buscar todos los apellidos usando _person-group/name/surname_ y sacar el texto que este tiene

In [ ]:
def get_text(node, key):
    elem = node.find(key)
    result = elem.text if elem is not None else None
    return result

def get_attr(node, key):
    if key in node:
        result = node.get(key)
    else:
        result = None
    return result

references = [
    dict(
        [
            ('citation-type', get_attr(k, 'citation-type')),                # paso 3
            ('article-title', get_text(k, 'article-title')),                # paso 2
            ('year', get_text(k, 'year')),                                  # paso 2
            ('source', get_text(k, 'source')),                              # paso 2
        ] +
        [
            (f'author_{idx}', j.text) 
            for idx, j in enumerate(k.findall('person-group/name/surname')) # paso 4
        ])
    for k in root.findall('back/ref-list/ref/nlm-citation')]                # paso 1

In [ ]:
df = pd.DataFrame(references)
df.head()

,citation-type,article-title,year,source,author_0,author_1,author_2,author_3,author_4,author_5,author_6
0,None,Las universidades privadas a los quince años (...,1997,Informe sobre la educación superior en Chile 1997,ALBORNOZ,LORCA,PERSICO,NaN,NaN,NaN,NaN
1,None,Ingreso a la Educación Superior: El Sistema Na...,1985,Cuadernos del Consejo de Rectores de las Unive...,ARANDA,NaN,NaN,NaN,NaN,NaN,NaN
2,None,Variables que inciden en el aprendizaje de los...,1997,"Educación, eficiencia y equidad.",ARRIAGADA,COHEN,NaN,NaN,NaN,NaN,NaN
3,None,None,1991,Informe de los resultados de las pruebas de ad...,AVILA,NaN,NaN,NaN,NaN,NaN,NaN
4,None,None,1993,Modificaciones propuestas para la sección Verb...,AVILA,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Guardar el resultado en CSV
df.to_csv('reseult_paper_parser.csv', index=False)

Por ultimo si esta muy apurado puedes subir todo los xml acá, vas al icono de carpeta que esta a la derecha y le das upload a todos los archivos y despues corres el codigo de abajo y te va a generar un csv por cada xml

In [ ]:
import os 

import pandas as pd
import xml.etree.ElementTree as ET

def get_text(node, key):
    elem = node.find(key)
    result = elem.text if elem is not None else None
    return result

def get_attr(node, key):
    if key in node:
        result = node.get(key)
    else:
        result = None
    return result

def extract_info(path_xml):
    root = ET.parse(path_xml).getroot()
    references = [
        dict(
            [
                ('citation-type', get_attr(k, 'citation-type')),                # paso 3
                ('article-title', get_text(k, 'article-title')),                # paso 2
                ('year', get_text(k, 'year')),                                  # paso 2
                ('source', get_text(k, 'source')),                              # paso 2
            ] +
            [
                (f'author_{idx}', j.text) 
                for idx, j in enumerate(k.findall('person-group/name/surname')) # paso 4
            ])
        for k in root.findall('back/ref-list/ref/nlm-citation')]                # paso 1
    df = pd.DataFrame(references)
    return df


file_path = './'
files = [k for k in os.listdir(file_path) if k.endswith('.xml')]

for file in files:
    source_file = os.path.join(file_path, file)
    destination_file = os.path.join(file_path, file.replace('.xml', '.csv'))
    df = extract_info(source_file)    
    df.to_csv(destination_file)
    print(f'{file} ok')

1.xml ok


['1.xml']